In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'/home/nistoroboto/software/NistoRoboto/git/src/')

In [3]:
import numpy as np 
import ipywidgets
import matplotlib.pyplot as plt
import requests
import time
import json
import itertools 
import random

# Initialize the OT2 Deck

In [27]:
import NistoRoboto
from NistoRoboto.prep.Mixture import Mixture
from NistoRoboto.prep.Component import Component
from NistoRoboto.prep.Deck import Deck
from NistoRoboto.shared.exceptions import MixingException

In [5]:
D2O = Component('D2O',mass=100.0,density=1.11)
salt = Component('salt',mass=1.0)
SDS = Component('SDS',mass=1.0)
MW_NaCl = 58.44 #g/mol

In [6]:
stock_D2O = Mixture([D2O])

stock_SDS = Mixture([SDS,D2O])
stock_SDS['SDS'].mass = 11.1024

stock_salt = Mixture([salt,D2O])
stock_salt['salt'].mass = 2.670

In [7]:
target_solution = Mixture([D2O,SDS,salt])
target_solution.mass = 1.6

In [8]:
phi_SDS_list = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]
phi_salt_list = [0.0,0.001,0.002,0.003,0.004,0.005]
phi_list = []
for phi_SDS in phi_SDS_list:
    for phi_salt in phi_salt_list:
        phi_D2O = 1.0-phi_SDS-phi_salt
        phi_list.append((phi_D2O,phi_SDS,phi_salt))
phi_list = phi_list*2

In [9]:
deck =  Deck()
tipracks=[(10,'opentrons_96_tiprack_1000ul'),(11,'opentrons_96_tiprack_1000ul')]
deck.add_pipette('p1000_single','left',tipracks=tipracks)

tipracks=[(8,'opentrons_96_tiprack_300ul'),(9,'opentrons_96_tiprack_300ul')]
deck.add_pipette('p300_single','right',tipracks=tipracks)

In [10]:
deck.add_container('opentrons_24_aluminumblock_nest_1.5ml_snapcap',1)
deck.add_container('opentrons_24_aluminumblock_nest_1.5ml_snapcap',2)
deck.add_container('opentrons_24_aluminumblock_nest_1.5ml_snapcap',4)
deck.add_container('opentrons_24_aluminumblock_nest_1.5ml_snapcap',5)
deck.add_container('nist_2_100ml_poly_bottle',3)
deck.add_container('nist_2_100ml_poly_bottle',6)
deck.add_loader ('nist_1_10ml_syringeloader',7)

In [11]:
deck.add_stock(stock_D2O,'3A1')
deck.add_stock(stock_D2O,'3A2')
deck.add_stock(stock_salt,'6A1')
deck.add_stock(stock_SDS,'6A2')

In [12]:
target_solution = Mixture([D2O,SDS,salt])
target_solution.mass = 1.6

phi_D2O = 0.95
phi_SDS = 0.045
phi_salt = 0.005
target_solution.set_mass_fractions({'D2O':phi_D2O,'SDS':phi_SDS,'salt':phi_salt})
deck.reset_targets()
deck.add_target(target_solution,'1A1')
deck.make_protocol(volume_cutoff=0.03)
for p in deck.protocol:
    print(p)


<PipetteAction Vol:257.597 3A1-->1A1>
<PipetteAction Vol:257.597 3A2-->1A1>
<PipetteAction Vol:269.933 6A1-->1A1>
<PipetteAction Vol:584.242 6A2-->1A1>


In [13]:
deck.make_script('align.py')

In [14]:
deck.reset_stocks()
deck.add_stock(stock_D2O,'3A1')
deck.add_stock(stock_D2O,'3A2')
deck.add_stock(stock_salt,'6A1')
deck.add_stock(stock_SDS,'6A2')

In [15]:
deck.init_remote_connection('10.42.0.30',home=True)
deck.send_deck_config()

# Intialize the Nice client

In [16]:
import nice
nice_client = nice.connect(host='NGBSANS.ncnr.nist.gov')
import NiceLib

In [17]:
def measure_sans(client,params,runGroup=10,prefix='ROBOT',user='NGB'):
    params_str = json.dumps(params).replace(':','=')
    cmd = f'runPoint {params_str} -g {runGroup} -p \"{prefix}\" -u \"{user}\"'
    print(cmd)
    client.console(cmd)
    # client.console_wait(cmd)

## Initialize the Cell Client

In [18]:
from NistoRoboto.DeviceServer.Client import Client
cell_client = Client('localhost')
cell_client.login('CellClient')


# Run Measurement

In [24]:
nice_params ={
'counter.countAgainst':'TIME',
'counter.timePreset':'300.0', 
'configuration':'1p15m 5A Offset Scatt',
'groupid':'21', 
'filePurpose':'SCATTERING', 
'intent':'Sample', 
'sample.description':'Test', 
'sample.thickness':'1.0', 
    
} 

sans_configs = []
sans_configs.append(('1p15m 5A Offset',300))
sans_configs.append(('5p2m 5A',600))
sans_configs.append(('5p2m 12A',1200))

In [21]:
blocks = [1,2,4,5]
cols = range(6)
rows = ['A','B','C','D']
target_locs = []
for block in blocks:
    for r in rows:
        for c in cols:
            target_locs.append(str(block)+r+str(c+1))
        
target_locs

['1A1',
 '1A2',
 '1A3',
 '1A4',
 '1A5',
 '1A6',
 '1B1',
 '1B2',
 '1B3',
 '1B4',
 '1B5',
 '1B6',
 '1C1',
 '1C2',
 '1C3',
 '1C4',
 '1C5',
 '1C6',
 '1D1',
 '1D2',
 '1D3',
 '1D4',
 '1D5',
 '1D6',
 '2A1',
 '2A2',
 '2A3',
 '2A4',
 '2A5',
 '2A6',
 '2B1',
 '2B2',
 '2B3',
 '2B4',
 '2B5',
 '2B6',
 '2C1',
 '2C2',
 '2C3',
 '2C4',
 '2C5',
 '2C6',
 '2D1',
 '2D2',
 '2D3',
 '2D4',
 '2D5',
 '2D6',
 '4A1',
 '4A2',
 '4A3',
 '4A4',
 '4A5',
 '4A6',
 '4B1',
 '4B2',
 '4B3',
 '4B4',
 '4B5',
 '4B6',
 '4C1',
 '4C2',
 '4C3',
 '4C4',
 '4C5',
 '4C6',
 '4D1',
 '4D2',
 '4D3',
 '4D4',
 '4D5',
 '4D6',
 '5A1',
 '5A2',
 '5A3',
 '5A4',
 '5A5',
 '5A6',
 '5B1',
 '5B2',
 '5B3',
 '5B4',
 '5B5',
 '5B6',
 '5C1',
 '5C2',
 '5C3',
 '5C4',
 '5C5',
 '5C6',
 '5D1',
 '5D2',
 '5D3',
 '5D4',
 '5D5',
 '5D6']

In [25]:
cell_client.wait()

In [26]:
load_loc = '7A1'
random.shuffle(phi_list)

for mfrac in phi_list:
    loc = target_locs.pop()
    phi_D2O,phi_SDS,phi_salt = mfrac
    target_solution.set_mass_fractions({'D2O':phi_D2O,'SDS':phi_SDS,'salt':phi_salt})
        
    print(f'--> Making protocol for sample {mfrac}')
    deck.reset_targets()
    deck.add_target(target_solution,loc)
    try:
        deck.make_protocol(volume_cutoff=0.03)
    except (MixingException, RuntimeError):
        print(f'--> Skipping {mfrac} due to protocol error')
        continue
    
    print(f'--> Making and loading sample {mfrac}')
    deck.init_remote_connection('10.42.0.30')
    deck.send_protocol()
    deck.load_sample(1000,loc,load_loc)
    deck.client.wait() #wait for queue to empty
      
    print(f'--> Loading sample {mfrac} into flow-cell')
    cell_client.login('CellClient')
    cell_client.debug(state=False)
    cell_client.enqueue(task_name='loadSample')
    cell_client.wait()
    
    for config,count in sans_configs:
        print(f'--> Asking NICE to measure sample {mfrac}')
        nice_params['configuration']      = config + ' Scatt'
        nice_params['filePurpose']        = f'SCATTERING'
        nice_params['counter.timePreset'] = str(count)
        nice_params['sample.description'] = f'D2O={mfrac[0]:4.3f} SDS={mfrac[1]:4.3f} NaCl={mfrac[2]:4.3f} {nice_params["configuration"]}'
        measure_sans(nice_client,nice_params)
            
        print(f'--> Asking NICE to measure transmission of {mfrac}')
        nice_params['configuration']      = config + ' Trans'
        nice_params['filePurpose']        = f'TRANSMISSION'
        nice_params['counter.timePreset'] = '180'
        nice_params['sample.description'] = f'D2O={mfrac[0]:4.3f} SDS={mfrac[1]:4.3f} NaCl={mfrac[2]:4.3f} {nice_params["configuration"]}'
        measure_sans(nice_client,nice_params)
        
    print(f'--> Waiting for NICE to measure {mfrac}')
    time.sleep(60)
    while str(nice_client.queue.queue_state) != 'IDLE':
        time.sleep(5)
        
    print(f'--> Cleaning up sample {mfrac}')
    cell_client.login('CellClient')
    cell_client.debug(state=False)
    cell_client.enqueue(task_name='rinseSyringe')
    cell_client.enqueue(task_name='rinseCellFlood')
    cell_client.enqueue(task_name='rinseCatch')
    cell_client.enqueue(task_name='rinseSyringe')
    cell_client.wait()
    
    print(f'--> All done for {mfrac}')

--> Making protocol for sample (0.986, 0.01, 0.004)
--> Making and loading sample (0.986, 0.01, 0.004)
--> Loading sample (0.986, 0.01, 0.004) into flow-cell
--> Asking NICE to measure sample (0.986, 0.01, 0.004)
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "300", "configuration"= "1p15m 5A Offset Scatt", "groupid"= "21", "filePurpose"= "SCATTERING", "intent"= "Sample", "sample.description"= "D2O=0.986 SDS=0.010 NaCl=0.004 1p15m 5A Offset Scatt", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
--> Asking NICE to measure transmission of (0.986, 0.01, 0.004)
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "1p15m 5A Offset Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "D2O=0.986 SDS=0.010 NaCl=0.004 1p15m 5A Offset Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
--> Asking NICE to measure sample (0.986, 0.01, 0.004)
runPoint {"counter.countAgainst"= "TIME", "cou

NameError: name 'MixingException' is not defined

# Backup cleaning

In [20]:
cell.rinseSyringe()
cell.rinseCellFlood()
cell.rinseCatch()
cell.rinseSyringe()

In [28]:
cell.rinseCatch()

# Measure Empty and Blocked

In [28]:
nice_params['configuration'] = f'5p2m 12A Scatt'
nice_params['filePurpose'] = f'SCATTERING'
nice_params['counter.timePreset'] = '300'
nice_params['sample.description'] = f'Empty Cell {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_params['configuration'] = f'5p2m 12A Trans'
nice_params['filePurpose'] = f'TRANSMISSION'
nice_params['counter.timePreset'] = '180'
nice_params['sample.description'] = f'Empty Cell {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_params['configuration'] = f'5p2m 5A Trans'
nice_params['filePurpose'] = f'TRANSMISSION'
nice_params['counter.timePreset'] = '180'
nice_params['sample.description'] = f'Empty Cell {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_params['configuration'] = f'5p2m 5A Scatt'
nice_params['filePurpose'] = f'SCATTERING'
nice_params['counter.timePreset'] = '300'
nice_params['sample.description'] = f'Empty Cell {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_params['configuration'] = f'1p15m 5A Offset Scatt'
nice_params['filePurpose'] = f'SCATTERING'
nice_params['counter.timePreset'] = '300'
nice_params['sample.description'] = f'Empty Cell {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)



runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "300", "configuration"= "5p2m 12A Scatt", "groupid"= "21", "filePurpose"= "SCATTERING", "intent"= "Sample", "sample.description"= "Empty Cell 5p2m 12A Scatt", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "5p2m 12A Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "Empty Cell 5p2m 12A Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "5p2m 5A Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "Empty Cell 5p2m 5A Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "300", "configuration"= "5p2m 5A Scatt", "groupid"= "21", "filePurpose"= "SCATTERING", "intent"= "Sample",

In [29]:
nice_client.console('move chamberTranslation 0')

nice_params['configuration'] = f'5p2m 12A Trans'
nice_params['filePurpose'] = f'TRANSMISSION'
nice_params['counter.timePreset'] = '180'
nice_params['sample.description'] = f'Open Beam {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_params['configuration'] = f'5p2m 5A Trans'
nice_params['filePurpose'] = f'TRANSMISSION'
nice_params['counter.timePreset'] = '180'
nice_params['sample.description'] = f'Open Beam {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)


nice_params['configuration'] = f'1p15m 1A Offset Trans'
nice_params['filePurpose'] = f'TRANSMISSION'
nice_params['counter.timePreset'] = '180'
nice_params['sample.description'] = f'Open Beam {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

nice_client.console('move chamberTranslation -34.9982')


runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "5p2m 12A Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "Open Beam 5p2m 12A Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "5p2m 5A Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "Open Beam 5p2m 5A Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "180", "configuration"= "1p15m 1A Offset Trans", "groupid"= "21", "filePurpose"= "TRANSMISSION", "intent"= "Sample", "sample.description"= "Open Beam 1p15m 1A Offset Trans", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"


In [33]:
nice_params['configuration'] = f'1p15m 5A Offset Scatt'
nice_params['filePurpose'] = f'SCATTERING'
nice_params['counter.timePreset'] = '300'
nice_params['sample.description'] = f'Blocked Beam {nice_params["configuration"]}'
measure_sans(nice_client,nice_params)

runPoint {"counter.countAgainst"= "TIME", "counter.timePreset"= "300", "configuration"= "1p15m 5A Offset Scatt", "groupid"= "21", "filePurpose"= "SCATTERING", "intent"= "Sample", "sample.description"= "Blocked Beam 1p15m 5A Offset Scatt", "sample.thickness"= "1.0"} -g 10 -p "ROBOT" -u "NGB"
